####**Задание 1.**

In [1]:
import numpy as np
import pandas as pd


class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon*self.prior)

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        speaker = self.speaker().T
        return rownorm(speaker * self.prior)


def rownorm(mat):
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Example from the class slides"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses']
    states = ['r1', 'r2']
    lex = pd.DataFrame([
        [0.0, 1.0],
        [1.0, 1.0]], index=msgs, columns=states)

    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.5, 0.5], costs=[0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,costs
hat,0.0,1.0,0.0
glasses,1.0,1.0,0.0
prior,0.5,0.5,
alpha,1.0,,



Literal listener


,r1,r2
hat,0.0,1.0
glasses,0.5,0.5



Pragmatic speaker


,hat,glasses
r1,0.000000,1.000000
r2,0.666667,0.333333



Pragmatic listener


,r1,r2
hat,0.00,1.00
glasses,0.75,0.25


####**Задание 2.**

Да, из таблицы прагматического слушающего можно вывести скалярную импликатуру: если необходимо указать человека с шляпой, между r1 и r2 будет выбран r1, потому что в строке "шляпа" значение 0.75 больше, чем 0.25 (у r3 нет шляпы, значение 0, он точно проигрывает).

In [2]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
    [0.0, 1.0, 0.0]], index=msgs, columns=states)

basic_mod = RSA(lexicon=lex, prior=[0.1, 0.45, 0.45], costs=[0.0, 0.0, 0.0])
display_reference_game(basic_mod)
print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
hat,1.0,1.0,0.0,0.0
glasses,0.0,0.0,1.0,0.0
moustache,0.0,1.0,0.0,0.0
prior,0.1,0.45,0.45,
alpha,1.0,,,



Literal listener


,r1,r2,r3
hat,0.181818,0.818182,0.0
glasses,0.000000,0.000000,1.0
moustache,0.000000,1.000000,0.0



Pragmatic speaker


,hat,glasses,moustache
r1,1.00,0.0,0.00
r2,0.45,0.0,0.55
r3,0.00,1.0,0.00



Pragmatic listener


,r1,r2,r3
hat,0.330579,0.669421,0.0
glasses,0.000000,0.000000,1.0
moustache,0.000000,1.000000,0.0


Поменяли исходные вероятности - теперь для поиска агента со шляпой выводится другая скалярная импликатура и побеждает r2, а не r1 как в первом случае, потому что теперь им соответсвуют значения 0.33 и 0.67 соответственно. Это объясняется тем, что исходная вероятность у r2 была 0.45, а у r1 - всего 0.1.

####**Задание 3.**

**Условия:** исходные вероятности 0.35, 0.35, 0.3, цена высказывания: 0, 0, 0.

In [3]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [0.0, 1.0, 0.0]], index=msgs, columns=states)

basic_mod = RSA(lexicon=lex, prior=[0.35, 0.35, 0.3], costs=[0.0, 0.0, 0.0])
display_reference_game(basic_mod)
print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
hat,1.00,1.0,0.0,0.0
glasses,0.00,1.0,1.0,0.0
moustache,0.00,1.0,0.0,0.0
prior,0.35,0.35,0.3,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.500000,0.000000
glasses,0.0,0.538462,0.461538
moustache,0.0,1.000000,0.000000



Pragmatic speaker


,hat,glasses,moustache
r1,1.000000,0.000000,0.000000
r2,0.245283,0.264151,0.490566
r3,0.000000,1.000000,0.000000



Pragmatic listener


,r1,r2,r3
hat,0.80303,0.196970,0.000000
glasses,0.00000,0.235577,0.764423
moustache,0.00000,1.000000,0.000000


В результате получились две скалярные импликатуры: выбор r1 для шляпы и r3 для очков. Это также соответствует моей лингвистически-прагматической интуиции.

Пусть теперь стоимость высказывания "шляпа" будет отрицательная, а стоимость высказывания "очки" будет небольшая положительная.

In [4]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [0.0, 1.0, 0.0]], index=msgs, columns=states)

basic_mod = RSA(lexicon=lex, prior=[0.35, 0.35, 0.3], costs=[-10, 2, 0.0])
display_reference_game(basic_mod)
print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
hat,1.00,1.0,0.0,-10.0
glasses,0.00,1.0,1.0,2.0
moustache,0.00,1.0,0.0,0.0
prior,0.35,0.35,0.3,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.500000,0.000000
glasses,0.0,0.538462,0.461538
moustache,0.0,1.000000,0.000000



Pragmatic speaker


,hat,glasses,moustache
r1,1.000000,0.000000,0.000000
r2,0.000005,0.799142,0.200854
r3,0.000000,1.000000,0.000000



Pragmatic listener


,r1,r2,r3
hat,0.999995,0.000005,0.000000
glasses,0.000000,0.482491,0.517509
moustache,0.000000,1.000000,0.000000


Теперь скалярная импликатура стала одна: для шляпы выбирается r1 (со значением, близким к 1, так как мы поменяли стоимости высказываний). Для очков выбор между r2 и r3 теперь затруднен, потому что их значения различаются незначительно.

Теперь поменяем стоимость высказываний так, чтобы импликатуры снова стало 2.

In [6]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [0.0, 1.0, 0.0]], index=msgs, columns=states)

basic_mod = RSA(lexicon=lex, prior=[0.35, 0.35, 0.3], costs=[3, 3, 0.0])
display_reference_game(basic_mod)
print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
hat,1.00,1.0,0.0,3.0
glasses,0.00,1.0,1.0,3.0
moustache,0.00,1.0,0.0,0.0
prior,0.35,0.35,0.3,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.500000,0.000000
glasses,0.0,0.538462,0.461538
moustache,0.0,1.000000,0.000000



Pragmatic speaker


,hat,glasses,moustache
r1,1.000000,0.000000,0.00000
r2,0.459454,0.494796,0.04575
r3,0.000000,1.000000,0.00000



Pragmatic listener


,r1,r2,r3
hat,0.685188,0.314812,0.00000
glasses,0.000000,0.365990,0.63401
moustache,0.000000,1.000000,0.00000


Теперь для шляпы выбирается r1, для очков выбирается r3, как было и с нулевыми стоимостями всех высказываний. Однако в данном случае значения "победителей" меньше (хотя стоимость высказываний "шляпа" и "очки" одинаковая).

In [7]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [0.0, 1.0, 0.0]], index=msgs, columns=states)

basic_mod = RSA(lexicon=lex, prior=[0.35, 0.35, 0.3], costs=[3, 3, 3])
display_reference_game(basic_mod)
print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
hat,1.00,1.0,0.0,3
glasses,0.00,1.0,1.0,3
moustache,0.00,1.0,0.0,3
prior,0.35,0.35,0.3,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.500000,0.000000
glasses,0.0,0.538462,0.461538
moustache,0.0,1.000000,0.000000



Pragmatic speaker


,hat,glasses,moustache
r1,1.000000,0.000000,0.000000
r2,0.245283,0.264151,0.490566
r3,0.000000,1.000000,0.000000



Pragmatic listener


,r1,r2,r3
hat,0.80303,0.196970,0.000000
glasses,0.00000,0.235577,0.764423
moustache,0.00000,1.000000,0.000000


А вот такая таблица полностью аналогична первой таблице с нулевыми стоимостями, потому что стоимости сами по себе (их аболютные значения) ни на что не влияют, важна разница в них.

Теперь добавим одному безусому агенту усы, чтобы было интереснее.

In [8]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [0.0, 1.0, 1.0]], index=msgs, columns=states)

basic_mod = RSA(lexicon=lex, prior=[0.35, 0.35, 0.3], costs=[3, 3, 3])
display_reference_game(basic_mod)
print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
hat,1.00,1.0,0.0,3
glasses,0.00,1.0,1.0,3
moustache,0.00,1.0,1.0,3
prior,0.35,0.35,0.3,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.500000,0.000000
glasses,0.0,0.538462,0.461538
moustache,0.0,0.538462,0.461538



Pragmatic speaker


,hat,glasses,moustache
r1,1.000000,0.000000,0.000000
r2,0.317073,0.341463,0.341463
r3,0.000000,0.500000,0.500000



Pragmatic listener


,r1,r2,r3
hat,0.759259,0.240741,0.000000
glasses,0.000000,0.443439,0.556561
moustache,0.000000,0.443439,0.556561


Логично, что строки с очками и усами полностью совпали.

In [9]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [0.0, 1.0, 1.0]], index=msgs, columns=states)

basic_mod = RSA(lexicon=lex, prior=[0.35, 0.35, 0.3], costs=[7, 7, 3])
display_reference_game(basic_mod)
print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
hat,1.00,1.0,0.0,7
glasses,0.00,1.0,1.0,7
moustache,0.00,1.0,1.0,3
prior,0.35,0.35,0.3,
alpha,1.00,,,



Literal listener


,r1,r2,r3
hat,0.5,0.500000,0.000000
glasses,0.0,0.538462,0.461538
moustache,0.0,0.538462,0.461538



Pragmatic speaker


,hat,glasses,moustache
r1,1.000000,0.000000,0.000000
r2,0.476952,0.513640,0.009408
r3,0.000000,0.982014,0.017986



Pragmatic listener


,r1,r2,r3
hat,0.67707,0.322930,0.000000
glasses,0.00000,0.378968,0.621032
moustache,0.00000,0.378968,0.621032


Изменили стоимости высказываний -- получили три скалярные импликатуры (r1 для шляпы, r3 для очков и усов). До этого ненулевые значения агентов в строках с очками и усами различались незначительно. Неудивительно также, что значения в строках "очки" и "усы" тут тоже идентичны.

In [13]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [0.0, 1.0, 1.0]], index=msgs, columns=states)

basic_mod = RSA(lexicon=lex, prior=[0.3, 0.6, 0.1], costs=[7, 7, 3])
display_reference_game(basic_mod)
print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
hat,1.0,1.0,0.0,7
glasses,0.0,1.0,1.0,7
moustache,0.0,1.0,1.0,3
prior,0.3,0.6,0.1,
alpha,1.0,,,



Literal listener


,r1,r2,r3
hat,0.333333,0.666667,0.000000
glasses,0.000000,0.857143,0.142857
moustache,0.000000,0.857143,0.142857



Pragmatic speaker


,hat,glasses,moustache
r1,1.000000,0.000000,0.000000
r2,0.433039,0.556764,0.010197
r3,0.000000,0.982014,0.017986



Pragmatic listener


,r1,r2,r3
hat,0.535884,0.464116,0.000000
glasses,0.000000,0.772819,0.227181
moustache,0.000000,0.772819,0.227181


Теперь мы изменили значения исходных вероятностей так, что про шляпу однозначный вывод сделать нельзя, и скалярной импликатуры нет (значения 0.53 и 0.46 близкие). Две скалярные импликатуры появляются для очков и усов, причём теперь выигрывает r2 в обоих случаях, потому что у него наибольшая исходная вероятность.

Пусть усы теперь есть у всех.

In [15]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [1.0, 1.0, 1.0]], index=msgs, columns=states)

basic_mod = RSA(lexicon=lex, prior=[0.3, 0.6, 0.1], costs=[7, 7, 3])
display_reference_game(basic_mod)
print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
hat,1.0,1.0,0.0,7
glasses,0.0,1.0,1.0,7
moustache,1.0,1.0,1.0,3
prior,0.3,0.6,0.1,
alpha,1.0,,,



Literal listener


,r1,r2,r3
hat,0.333333,0.666667,0.000000
glasses,0.000000,0.857143,0.142857
moustache,0.300000,0.600000,0.100000



Pragmatic speaker


,hat,glasses,moustache
r1,0.983783,0.000000,0.016217
r2,0.434367,0.558472,0.007160
r3,0.000000,0.987341,0.012659



Pragmatic listener


,r1,r2,r3
hat,0.531052,0.468948,0.000000
glasses,0.000000,0.772406,0.227594
moustache,0.466581,0.412016,0.121403


Скалярная импликатура для усов теперь не выводится, потому что значения для r1 и r2 очень близки. Аналогично не выводится точно импликатура для шляпы. Для очков побеждает r2.

In [19]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [1.0, 1.0, 1.0]], index=msgs, columns=states)

basic_mod = RSA(lexicon=lex, prior=[0.3, 0.6, 0.1], costs=[0, 0, 0])
display_reference_game(basic_mod)
print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
hat,1.0,1.0,0.0,0
glasses,0.0,1.0,1.0,0
moustache,1.0,1.0,1.0,0
prior,0.3,0.6,0.1,
alpha,1.0,,,



Literal listener


,r1,r2,r3
hat,0.333333,0.666667,0.000000
glasses,0.000000,0.857143,0.142857
moustache,0.300000,0.600000,0.100000



Pragmatic speaker


,hat,glasses,moustache
r1,0.526316,0.000000,0.473684
r2,0.313901,0.403587,0.282511
r3,0.000000,0.588235,0.411765



Pragmatic listener


,r1,r2,r3
hat,0.456033,0.543967,0.000000
glasses,0.000000,0.804557,0.195443
moustache,0.402806,0.480477,0.116717


Мы обнулили стоимости всех высказываний -- теперь для всех атрибутов по скалярной импликатуре побеждает r2, однако для шляпы разница в значении не слишком большая (хотя уже достаточная, наверное).

Теперь изменим параметр альфа.

In [22]:
msgs = ['hat', 'glasses', 'moustache']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [1.0, 1.0, 0.0],
    [0.0, 1.0, 1.0],
    [1.0, 1.0, 1.0]], index=msgs, columns=states)

basic_mod = RSA(lexicon=lex, prior=[0.3, 0.6, 0.1], costs=[0, 0, 0], alpha = 10)
display_reference_game(basic_mod)
print("\nLiteral listener")
display(basic_mod.literal_listener())

print("\nPragmatic speaker")
display(basic_mod.speaker())

print("\nPragmatic listener")
display(basic_mod.listener())

,r1,r2,r3,costs
hat,1.0,1.0,0.0,0
glasses,0.0,1.0,1.0,0
moustache,1.0,1.0,1.0,0
prior,0.3,0.6,0.1,
alpha,10.0,,,



Literal listener


,r1,r2,r3
hat,0.333333,0.666667,0.000000
glasses,0.000000,0.857143,0.142857
moustache,0.300000,0.600000,0.100000



Pragmatic speaker


,hat,glasses,moustache
r1,0.741467,0.000000,0.258533
r2,0.073033,0.901501,0.025465
r3,0.000000,0.972528,0.027472



Pragmatic listener


,r1,r2,r3
hat,0.835424,0.164576,0.000000
glasses,0.000000,0.847603,0.152397
moustache,0.811414,0.159846,0.028740


При альфа = 10 результаты получились совсем не похожими на предыдущие. Теперь по скалярным импликатурам r1 уверенно выигрывает для шляпы и усов, а r2 для очков.

####**Задание 4.**

Мне кажется, было бы интересно посмотреть на применение теории рационального речевого акта в исследованиях инкорпорирующих языков, например, чукотского. 

В чём исследовательский вопрос: в чукотском нет четких правил, по которым однозначно вычислялся бы именной участник, инкорпорирующийся в глагол. Цитата из хендаута Алексия Игоревича Виняра:

"Имя в глагол: инкорпорироваться может пациентивный участник (Sp или P); слабоодушевлённая и слабоагентивная Причина (S); Место/Источник/Цель и Инструмент (последние — не особо продуктивно)
-  Чем пациентивнее участник и чем менее он одушевлённый, тем больше мы ожидаем, что его будет возможно инкорпорироваться
- Инкорпорация SP и P участника может позволять другим участникам занять позицию Itr.Subj/DO и получить номинатив
- **Видимо — тем, которые более прагматически важные**
- Это может быть Обладатель части (тела); Бенефициант/Реципиент;
Цель/Источник/Место
- Основное требование: продвигаемый участник должен находиться под
(наиболее прямым) воздействием"

Идея для исследования в чукотском и в похожих ситуациях: соотнести результаты референциальных игр (вычислить) и предложения с инкорпорацией, описывающих тех же участников. Например, у нас есть те же люди в шляпе и/или очках. Как будут на чукотском в предложениях передаваться эти атрибуты в зависимости от конкретного набора? Возможно, инкорпорироваться будет самое прагматически информативное как раз. 

(Надо доформулировать бы, но вы поняли))